<h2>DATA CLEANING</h2>

Script ini digunakan untuk membersihkan data dari GrabPosisi jadi data yang lebih mudah untuk dimengerti dan dipelajari oleh model LSTMnya nanti. Proses Cleaning Data ini meliputi:
- Mengurutkan Data berdasarkan trj_id lalu pingtimestamp agar data terurut dengan baik
- Mengelompokkan data dan menjadikannya Pandas Multi Index Dataframe dengan trj_id sebagai index, untuk mempermudah pemisahan trajectories nantinya
- Me-resample data (Data awal memiliki sampling rate 1 kali per **detik**, namun beberapa kali jarak antar ping bisa lebih dari 1 detik). Setelah resampling, data memiliki sampling rate 1 kali per **menit** dan konstan (Tidak ada jarak antar ping yang berbeda)
<br/>
Hasil dari data cleaning ini sudah diexport menjadi clean_data.csv, sehingga script ini tidak perlu dijalankan kecuali ingin mengubah proses data cleaning di atas.

In [1]:
## Import libraries
import pandas as pd
from pathlib import Path

In [2]:
# Read Data
data_dir = Path('../GrabPosisi')
full_df = pd.DataFrame()
for parquet_file in data_dir.glob("*.parquet"):
    print(f"Adding file: {parquet_file}")
    full_df = pd.concat([full_df, pd.read_parquet(parquet_file)], ignore_index=True)

Adding file: ..\GrabPosisi\part-00000-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00001-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00002-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00003-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00004-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00005-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00006-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00007-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00008-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet
Adding file: ..\GrabPosisi\part-00009-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet


In [3]:
# Remove unnecessary columns (driving_mode, osname, accuracy)
train_df = full_df[['trj_id', 'pingtimestamp', 'rawlat', 'rawlng', 'speed', 'bearing']]

# Sort by trj_id and then pingtimestamp
train_df = train_df.sort_values(by=["trj_id", "pingtimestamp"])

print(train_df)

         trj_id  pingtimestamp    rawlat      rawlng  speed  bearing
29091989      1     1554992255 -6.197622  106.769017   5.58      180
29074360      1     1554992256 -6.197667  106.769007   5.33      177
10694992      1     1554992257 -6.197713  106.769012   5.43      177
10676471      1     1554992258 -6.197764  106.769020   5.84      178
22141074      1     1554992259 -6.197809  106.769018   5.28      179
...         ...            ...       ...         ...    ...      ...
19921434   9999     1555822630 -6.178844  106.841960   0.00        0
7604364    9999     1555822631 -6.178844  106.841960   0.00        0
25980783   9999     1555822632 -6.178844  106.841961   0.00        0
14850195   9999     1555822634 -6.178845  106.841963   0.00        0
19919728   9999     1555822635 -6.178845  106.841964   0.00        0

[55988420 rows x 6 columns]


In [4]:
# Remove data with 0 values
cleaned_df = train_df.dropna(subset=["speed", "pingtimestamp", "rawlat", "rawlng"]).query("pingtimestamp != 0 and rawlat != 0 and rawlng != 0")

print(cleaned_df)

         trj_id  pingtimestamp    rawlat      rawlng  speed  bearing
29091989      1     1554992255 -6.197622  106.769017   5.58      180
29074360      1     1554992256 -6.197667  106.769007   5.33      177
10694992      1     1554992257 -6.197713  106.769012   5.43      177
10676471      1     1554992258 -6.197764  106.769020   5.84      178
22141074      1     1554992259 -6.197809  106.769018   5.28      179
...         ...            ...       ...         ...    ...      ...
19921434   9999     1555822630 -6.178844  106.841960   0.00        0
7604364    9999     1555822631 -6.178844  106.841960   0.00        0
25980783   9999     1555822632 -6.178844  106.841961   0.00        0
14850195   9999     1555822634 -6.178845  106.841963   0.00        0
19919728   9999     1555822635 -6.178845  106.841964   0.00        0

[55988420 rows x 6 columns]


In [5]:
# Group data by trj_id
cleaned_df['pingtimestamp'] = pd.to_datetime(cleaned_df['pingtimestamp'], unit='s') # Convert unix timestamp to Pandas DateTime
grouped_data = cleaned_df.groupby('trj_id') # Group the dataframe by trj_id

# Resample by minute
def resample_trajectory(data):
  """
  First resample the data to every one minute (1T) by making the pingtimestamp an index (turning it to a pandas multi index dataframe),
  then linear interpolate tha data (if a data is null, take average of the previous and next data),
  then convert the pingtimestamp from index back to column.

  The result is a pandas multi index dataframe with the trj_id as index, and pingtimestamp, rawlat, rawlng, speed, bearing as columns.
  """
  resampled_data = data.resample('1T', on='pingtimestamp')['rawlat', 'rawlng', 'speed', 'bearing'].mean().interpolate('linear').reset_index()
  return resampled_data

resampled_data = grouped_data.apply(resample_trajectory)

print(resampled_data)

                pingtimestamp    rawlat      rawlng      speed     bearing
trj_id                                                                    
1      0  2019-04-11 14:17:00 -6.198042  106.769008   4.322800  179.920000
       1  2019-04-11 14:18:00 -6.200972  106.769202   8.014167  173.233333
       2  2019-04-11 14:19:00 -6.205394  106.769768  10.116136  171.477273
       3  2019-04-11 14:20:00 -6.210496  106.771217   9.307667  156.683333
       4  2019-04-11 14:21:00 -6.214969  106.773830  10.103333  139.777778
...                       ...       ...         ...        ...         ...
9999   33 2019-04-21 04:53:00 -6.184174  106.843572   3.187241  229.396552
       34 2019-04-21 04:54:00 -6.182703  106.842868   2.684167  277.566667
       35 2019-04-21 04:55:00 -6.180504  106.842337   5.244333  326.850000
       36 2019-04-21 04:56:00 -6.178920  106.841976   0.843966   83.672414
       37 2019-04-21 04:57:00 -6.178846  106.841960   0.000000    0.000000

[1223918 rows x 5 column

In [6]:
# Make hour_of_day and day_of_week from pingtimestamp (Used so the model can better predict user movement from time of day and day of week)
resampled_data['minute_of_hour'] = resampled_data['pingtimestamp'].dt.minute
resampled_data['hour_of_day'] = resampled_data['pingtimestamp'].dt.hour
resampled_data['day_of_week'] = resampled_data['pingtimestamp'].dt.dayofweek
resampled_data = resampled_data.drop('pingtimestamp', axis=1) # Drop pingtimestamp as it is no longer needed
print(resampled_data)

             rawlat      rawlng      speed     bearing  minute_of_hour  \
trj_id                                                                   
1      0  -6.198042  106.769008   4.322800  179.920000              17   
       1  -6.200972  106.769202   8.014167  173.233333              18   
       2  -6.205394  106.769768  10.116136  171.477273              19   
       3  -6.210496  106.771217   9.307667  156.683333              20   
       4  -6.214969  106.773830  10.103333  139.777778              21   
...             ...         ...        ...         ...             ...   
9999   33 -6.184174  106.843572   3.187241  229.396552              53   
       34 -6.182703  106.842868   2.684167  277.566667              54   
       35 -6.180504  106.842337   5.244333  326.850000              55   
       36 -6.178920  106.841976   0.843966   83.672414              56   
       37 -6.178846  106.841960   0.000000    0.000000              57   

           hour_of_day  day_of_week  

In [29]:
resampled_data.loc['1001']

KeyError: '1001'

In [15]:
# Export data to CSV
resampled_data.to_csv('clean_data.csv')